In [1]:
import urllib.request
import os
import tarfile

In [ ]:
# 下載IMDB資料集
url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath = "aclImdb_v1.tar.gz"

if not os.path.isfile(filepath):
    result = urllib.request.urlretrieve(url, filepath)
    print("downloaded:", result)

In [3]:
if not os.path.exists("data/aclImdb"):
    tfile = tarfile.open("aclImdb_v1.tar.gz", "r:gz")
    result = tfile.extractall("data/")

# 1. Import Library

In [4]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer

2024-10-26 14:56:49.393152: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-26 14:56:49.412748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1729925809.434258   94714 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1729925809.438405   94714 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-26 14:56:49.459234: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [5]:
import re


# 刪除文字中的HTML標簽
def rm_tags(text):
    re_tag = re.compile(r"<[^>]+>")
    return re_tag.sub("", text)

In [6]:
import os


# 讀取IMDB文件
def read_files(filetype):
    path = "data/aclImdb/"
    file_list = []

    positive_path = path + filetype + "/pos/"
    for f in os.listdir(positive_path):
        file_list += [positive_path + f]

    negative_path = path + filetype + "/neg/"
    for f in os.listdir(negative_path):
        file_list += [negative_path + f]

    print("read", filetype, "files:", len(file_list))

    all_labels = [1] * 12500 + [0] * 12500

    all_texts = []
    for fi in file_list:
        with open(fi, encoding="utf8") as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]

    return all_labels, all_texts

In [7]:
y_train, train_text = read_files("train")

read train files: 25000


In [8]:
y_test, test_text = read_files("test")

read test files: 25000


In [9]:
# 檢視評論內容
train_text[0]

'Fantastic movie. One to excite all 5 senses. Is not a true historical report and not all information is to be taken as factual information. True Hollywood conventions used, like playing A list and VERY attractive actors as the \'heroes\', such as Naomi Watts (Julia Cook - Ned Kelly\'s lover), Heath Ledger (Ned) and Orlando Bloom (Joe Byrne - Ned\'s right hand man), and unattractive (sorry Geoffrey Rush) actors play the drunken and corrupt Victorian Police Force. This also instills a very unreliable love story into the mix between Ned (Ledger) and Julia Cook (Watts) to entice all the romantics, females being especially susceptible. Even from the first scene, when Ned saves the fat youth from drowning and his dad calls him "sunshine" and had a "glint in his eye as he looked down at me, his hand on me shoulder," it is very romanticized and persuades viewers to side with Ned Kelly, the underdog. Besides, don\'t all Aussies love an underdog?'

In [10]:
# 評論結果
y_train[0]

1

In [11]:
train_text[12499]

"I agree that Capital City should be on DVD. I watched this show only by accident in 1994 and fell in love with Rolf Saxon as Hudson Talbot. It was nice to see Americans who work abroad in London in the financial industry for a change. I loved Rolf in this role and loved every other role that he has been in. I can't believe the show only lasted 13 episodes. I liked William Armstrong as Hudson's flamboyant charming friend in the series. When they aired this show in the New York City area, it was always late at night or at off times. The show is less than an hour long. I felt this show should have gone on longer but the casting changes in the second season really made the show a little less interesting. I didn't care for Sylvia but missed the actress, Julia Phillips-Lane in the previous season. I felt this show took chances and often it worked. It showed Americans who loved and chose to live in London. The American characters were not arrogant or tried to outdo their British counterparts

In [12]:
y_train[12499]

1

In [13]:
# 先讀取所有文章並建立字典，限制字典的數量nb_words=2000
# 建立token
token = Tokenizer(num_words=2000)
token.fit_on_texts(train_text)

In [14]:
# 查文章數量
print(token.document_count)

25000


In [15]:
print(token.word_index)

{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'is': 6, 'in': 7, 'it': 8, 'i': 9, 'this': 10, 'that': 11, 'was': 12, 'as': 13, 'for': 14, 'with': 15, 'movie': 16, 'but': 17, 'film': 18, 'on': 19, 'not': 20, 'you': 21, 'are': 22, 'his': 23, 'have': 24, 'be': 25, 'he': 26, 'one': 27, 'all': 28, 'at': 29, 'by': 30, 'an': 31, 'they': 32, 'who': 33, 'so': 34, 'from': 35, 'like': 36, 'her': 37, 'or': 38, 'just': 39, 'about': 40, "it's": 41, 'out': 42, 'has': 43, 'if': 44, 'some': 45, 'there': 46, 'what': 47, 'good': 48, 'more': 49, 'when': 50, 'very': 51, 'up': 52, 'no': 53, 'time': 54, 'she': 55, 'even': 56, 'my': 57, 'would': 58, 'which': 59, 'only': 60, 'story': 61, 'really': 62, 'see': 63, 'their': 64, 'had': 65, 'can': 66, 'were': 67, 'me': 68, 'well': 69, 'than': 70, 'we': 71, 'much': 72, 'been': 73, 'get': 74, 'bad': 75, 'will': 76, 'also': 77, 'do': 78, 'into': 79, 'people': 80, 'other': 81, 'first': 82, 'great': 83, 'because': 84, 'how': 85, 'him': 86, 'most': 87, "don't": 88, 'made